THIS CODE WAS INSIPIRED FROM https://github.com/WJMatthew/WESAD

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats 
import scipy.signal as scisig
import scipy.stats
import cvxEDA

ModuleNotFoundError: No module named 'cvxEDA'

In [2]:
# dir(cvxEDA)

In [3]:
fs_dict = {'ACC': 32, 'BVP': 64, 'EDA': 4, 'TEMP': 4, 'label': 700, 'Resp': 700, 'ECG':700}
WINDOW_IN_SECONDS = 1
label_dict = {'baseline': 0, 'stress': 1, 'amusement': 2, 'meditation':3}
int_to_label = {0: 'baseline', 1: 'stress', 2: 'amusement', 3:'meditation'}
feat_names = None
savePath = 'fresh_start'
subject_feature_path = '/subject_feats'

if not os.path.exists(savePath):
    os.makedirs(savePath)
if not os.path.exists(savePath + subject_feature_path):
    os.makedirs(savePath + subject_feature_path)


In [4]:
# cvxEDA
def eda_stats(y):
    Fs = fs_dict['EDA']
    yn = (y - y.mean()) / y.std()
    [r, p, t, l, d, e, obj] = cvxEDA.cvxEDA(yn, 1. / Fs)
    return [r, p, t, l, d, e, obj]


In [5]:
class SubjectData:

    def __init__(self, main_path, subject_number):
        self.name = f'S{subject_number}'
        self.subject_keys = ['signal', 'label', 'subject']
        self.signal_keys = ['chest', 'wrist']
        self.chest_keys = ['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp']
        self.wrist_keys = ['ACC', 'BVP', 'EDA', 'TEMP']
        with open(os.path.join(main_path, self.name) + '/' + self.name + '.pkl', 'rb') as file:
            self.data = pickle.load(file, encoding='latin1')
        self.labels = self.data['label']

    def get_wrist_data(self):
        data = self.data['signal']['wrist']
        data.update({'Resp': self.data['signal']['chest']['Resp']})
        data.update({'ECG': self.data['signal']['chest']['ECG']})
        data.update({'EMG': self.data['signal']['chest']['EMG']})
        data.update({'c_EDA': self.data['signal']['chest']['EDA']})
        data.update({'c_Temp': self.data['signal']['chest']['Temp']})
        data.update({'c_ACC': self.data['signal']['chest']['ACC']})
        return data

    def get_chest_data(self):
        return self.data['signal']['chest']

    def extract_features(self):  # only wrist
        results = \
            {
                key: get_statistics(self.get_wrist_data()[key].flatten(), self.labels, key)
                for key in self.wrist_keys
            }
        return results

In [6]:
def butter_lowpass(cutoff, fs, order=5):
    # Filtering Helper functions
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = scisig.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a


def butter_lowpass_filter(data, cutoff, fs, order=5):
    # Filtering Helper functions
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = scisig.lfilter(b, a, data)
    return y

def get_slope(series):
    linreg = scipy.stats.linregress(np.arange(len(series)), series )
    slope = linreg[0]
    return slope

def get_window_stats(data, label=-1):
    mean_features = np.mean(data)
    std_features = np.std(data)
    min_features = np.amin(data)
    max_features = np.amax(data)

    features = {'mean': mean_features, 'std': std_features, 'min': min_features, 'max': max_features,
                'label': label}
    return features


def get_net_accel(data):
    return (data['ACC_x'] ** 2 + data['ACC_y'] ** 2 + data['ACC_z'] ** 2).apply(lambda x: np.sqrt(x))


def get_peak_freq(x):
    f, Pxx = scisig.periodogram(x, fs=8)
    psd_dict = {amp: freq for amp, freq in zip(Pxx, f)}
    peak_freq = psd_dict[max(psd_dict.keys())]
    return peak_freq

In [7]:
# https://github.com/MITMediaLabAffectiveComputing/eda-explorer/blob/master/AccelerometerFeatureExtractionScript.py
def filterSignalFIR(eda, cutoff=0.4, numtaps=64):
    f = cutoff / (fs_dict['ACC'] / 2.0)
    FIR_coeff = scisig.firwin(numtaps, f)

    return scisig.lfilter(FIR_coeff, 1, eda)


def compute_features(e4_data_dict, labels, norm_type=None):
    # Dataframes for each sensor type
    eda_df = pd.DataFrame(e4_data_dict['EDA'], columns=['EDA'])
    bvp_df = pd.DataFrame(e4_data_dict['BVP'], columns=['BVP'])
    acc_df = pd.DataFrame(e4_data_dict['ACC'], columns=['ACC_x', 'ACC_y', 'ACC_z'])
    temp_df = pd.DataFrame(e4_data_dict['TEMP'], columns=['TEMP'])
    label_df = pd.DataFrame(labels, columns=['label'])
    resp_df = pd.DataFrame(e4_data_dict['Resp'], columns=['Resp'])
    
    ecg_df = pd.DataFrame(e4_data_dict['ECG'], columns=['ECG'])
    emg_df = pd.DataFrame(e4_data_dict['EMG'], columns=['EMG'])
    c_temp_df = pd.DataFrame(e4_data_dict['c_Temp'], columns=['c_Temp'])
    c_acc_df = pd.DataFrame(e4_data_dict['c_ACC'], columns=['c_ACC_x', 'c_ACC_y', 'c_ACC_z'])
    
    # Filter EDA
    eda_df['EDA'] = butter_lowpass_filter(eda_df['EDA'], 1.0, fs_dict['EDA'], 6)
    
    # Filter ACM
    for _ in acc_df.columns:
        acc_df[_] = filterSignalFIR(acc_df.values)

    # Adding indices for combination due to differing sampling frequencies
    eda_df.index = [(1 / fs_dict['EDA']) * i for i in range(len(eda_df))]
    bvp_df.index = [(1 / fs_dict['BVP']) * i for i in range(len(bvp_df))]
    acc_df.index = [(1 / fs_dict['ACC']) * i for i in range(len(acc_df))]
    temp_df.index = [(1 / fs_dict['TEMP']) * i for i in range(len(temp_df))]
    label_df.index = [(1 / fs_dict['label']) * i for i in range(len(label_df))]
    resp_df.index = [(1 / fs_dict['Resp']) * i for i in range(len(resp_df))]
    
    ecg_df.index = [(1 / fs_dict['ECG']) * i for i in range(len(ecg_df))]
    emg_df.index = [(1 / fs_dict['ECG']) * i for i in range(len(emg_df))]          #ECG fz = EMG fz
    c_temp_df.index = [(1 / fs_dict['ECG']) * i for i in range(len(c_temp_df))]
    c_acc_df.index = [(1 / fs_dict['ECG']) * i for i in range(len(c_acc_df))]
    
    # Change indices to datetime
    eda_df.index = pd.to_datetime(eda_df.index, unit='s')
    bvp_df.index = pd.to_datetime(bvp_df.index, unit='s')
    temp_df.index = pd.to_datetime(temp_df.index, unit='s')
    acc_df.index = pd.to_datetime(acc_df.index, unit='s')
    label_df.index = pd.to_datetime(label_df.index, unit='s')
    resp_df.index = pd.to_datetime(resp_df.index, unit='s')
    
    ecg_df.index  = pd.to_datetime(ecg_df.index , unit='s')
    emg_df.index  = pd.to_datetime(emg_df.index , unit='s')
    c_temp_df.index  = pd.to_datetime(c_temp_df.index , unit='s')
    c_acc_df.index = pd.to_datetime(c_acc_df.index, unit='s')
    
    # New EDA features
    r, p, t, l, d, e, obj = eda_stats(eda_df['EDA'])
    eda_df['EDA_phasic'] = r
    eda_df['EDA_smna'] = p
    eda_df['EDA_tonic'] = t
    
   
        
    # Combined dataframe - not used yet
    df = eda_df.join(bvp_df, how='outer')
    df = df.join(temp_df, how='outer')
    df = df.join(acc_df, how='outer')
    df = df.join(resp_df, how='outer')
    df = df.join(label_df, how='outer')
    df = df.join(ecg_df, how='outer')
    df = df.join(emg_df, how='outer')
    df = df.join(c_temp_df, how='outer')
    df = df.join(c_acc_df, how='outer')
    df['label'] = df['label'].fillna(method='bfill')
    df.reset_index(drop=True, inplace=True)

    if norm_type is 'std':
        # std norm
        df = (df - df.mean()) / df.std()
    elif norm_type is 'minmax':
        # minmax norm
        df = (df - df.min()) / (df.max() - df.min())

    # Groupby
    grouped = df.groupby('label')
    baseline = grouped.get_group(1)
    stress = grouped.get_group(2)
    amusement = grouped.get_group(3)
    meditation = grouped.get_group(4)
    
#     print ("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
#     print (df)
#     print (grouped.groups.keys())
#     print ("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
    
    return grouped, baseline, stress, amusement, meditation


def get_samples(data, n_windows, label):
    global feat_names
    global WINDOW_IN_SECONDS

    samples = []
    # Using label freq (700 Hz) as our reference frequency due to it being the largest
    # and thus encompassing the lesser ones in its resolution.
    window_len = fs_dict['label'] * WINDOW_IN_SECONDS

    for i in range(n_windows):
        # Get window of data
        w = data[window_len * i: window_len * (i + 1)]

        # Add/Calc rms acc
        # w['net_acc'] = get_net_accel(w)
        w = pd.concat([w, get_net_accel(w)])
        #w.columns = ['net_acc', 'ACC_x', 'ACC_y', 'ACC_z', 'BVP',
          #           'EDA', 'EDA_phasic', 'EDA_smna', 'EDA_tonic', 'TEMP',
            #         'label']
        cols = list(w.columns)
        cols[0] = 'net_acc'
        w.columns = cols
        
        # Calculate stats for window
        wstats = get_window_stats(data=w, label=label)

        # Seperating sample and label
        x = pd.DataFrame(wstats).drop('label', axis=0)
        y = x['label'][0]
        x.drop('label', axis=1, inplace=True)

        if feat_names is None:
            feat_names = []
            for row in x.index:
                for col in x.columns:
                    feat_names.append('_'.join([row, col]))

        # sample df
        wdf = pd.DataFrame(x.values.flatten()).T
        wdf.columns = feat_names
        wdf = pd.concat([wdf, pd.DataFrame({'label': y}, index=[0])], axis=1)
        
        # More feats
        wdf['BVP_peak_freq'] = get_peak_freq(w['BVP'].dropna())
        wdf['TEMP_slope'] = get_slope(w['TEMP'].dropna())
        samples.append(wdf)

    return pd.concat(samples)



In [8]:
def make_patient_data(subject_id):
    global savePath
    global WINDOW_IN_SECONDS

    # Make subject data object for Sx
    subject = SubjectData(main_path='WESAD', subject_number=subject_id)

    # Empatica E4 data - now with resp
    e4_data_dict = subject.get_wrist_data()
#     e4_data_dict = subject.get_chest_data()

    # norm type
    norm_type = None

    # The 3 classes we are classifying
    grouped, baseline, stress, amusement, meditation = compute_features(e4_data_dict, subject.labels, norm_type)

    # print(f'Available windows for {subject.name}:')
    n_baseline_wdws = int(len(baseline) / (fs_dict['label'] * WINDOW_IN_SECONDS))
    n_stress_wdws = int(len(stress) / (fs_dict['label'] * WINDOW_IN_SECONDS))
    n_amusement_wdws = int(len(amusement) / (fs_dict['label'] * WINDOW_IN_SECONDS))
    n_meditation_wdws = int(len(meditation) / (fs_dict['label'] * WINDOW_IN_SECONDS))
    # print(f'Baseline: {n_baseline_wdws}\nStress: {n_stress_wdws}\nAmusement: {n_amusement_wdws}\n')

    #
    baseline_samples = get_samples(baseline, n_baseline_wdws, 0)
    # Downsampling
    # baseline_samples = baseline_samples[::2]
    stress_samples = get_samples(stress, n_stress_wdws, 1)
    amusement_samples = get_samples(amusement, n_amusement_wdws, 2)
    meditation_samples = get_samples(meditation, n_meditation_wdws, 3)

    all_samples = pd.concat([baseline_samples, stress_samples, amusement_samples,meditation_samples])
    all_samples = pd.concat([all_samples.drop('label', axis=1), pd.get_dummies(all_samples['label'])], axis=1)
    
    # Selected Features
    # all_samples = all_samples[['EDA_mean', 'EDA_std', 'EDA_min', 'EDA_max',
    #                          'BVP_mean', 'BVP_std', 'BVP_min', 'BVP_max',
    #                        'TEMP_mean', 'TEMP_std', 'TEMP_min', 'TEMP_max',
    #                        'net_acc_mean', 'net_acc_std', 'net_acc_min', 'net_acc_max',
    #                        0, 1, 2]]
    # Save file as csv (for now)
    all_samples.to_csv(f'{savePath}{subject_feature_path}/S{subject_id}_feats_4.csv')

    # Does this save any space?
    subject = None


In [9]:
def combine_files(subjects):
    df_list = []
    for s in subjects:
        df = pd.read_csv(f'{savePath}{subject_feature_path}/S{s}_feats_4.csv', index_col=0)
        df['subject'] = s
        df_list.append(df)

    df = pd.concat(df_list)
    
#     df.info()
    
    df['label'] = (df['0'].astype(str) + df['1'].astype(str) + df['2'].astype(str)+ df['3'].astype(str)).apply(lambda x: x.index('1'))
    
#     print (df)
    
#     print ("***************************8")
#     df['label']
#     print ("***************************8")

    df.drop(['0', '1', '2','3'], axis=1, inplace=True)

    df.reset_index(drop=True, inplace=True)

    df.to_csv(f'{savePath}/may14_feats4_chest.csv')

    counts = df['label'].value_counts()
    print('Number of samples per class:')
    print (counts)
    for label, number in zip(counts.index, counts.values):
        print(f'{int_to_label[label]}: {number}')


In [10]:
if __name__ == '__main__':

    subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
#     subject_ids = [2]
    for patient in subject_ids:
        print(f'Processing data for S{patient}...')
        make_patient_data(patient)

    combine_files(subject_ids)
    print('Processing complete.')

Processing data for S2...


FileNotFoundError: [Errno 2] No such file or directory: 'WESAD/S2/S2.pkl'

In [28]:
pre = pd.read_csv("fresh_start/may14_feats4_chest.csv")

In [29]:
pre['subject'].value_counts()

10    54
17    53
16    53
15    53
14    53
13    53
11    53
8     53
9     52
7     52
6     52
5     52
4     51
3     51
2     50
Name: subject, dtype: int64

In [30]:
pre['label'].value_counts()

0    312
3    208
1    175
2     90
Name: label, dtype: int64

In [31]:
pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785 entries, 0 to 784
Data columns (total 73 columns):
Unnamed: 0         785 non-null int64
net_acc_mean       785 non-null float64
net_acc_std        785 non-null float64
net_acc_min        785 non-null float64
net_acc_max        785 non-null float64
ACC_x_mean         785 non-null float64
ACC_x_std          785 non-null float64
ACC_x_min          785 non-null float64
ACC_x_max          785 non-null float64
ACC_y_mean         785 non-null float64
ACC_y_std          785 non-null float64
ACC_y_min          785 non-null float64
ACC_y_max          785 non-null float64
ACC_z_mean         785 non-null float64
ACC_z_std          785 non-null float64
ACC_z_min          785 non-null float64
ACC_z_max          785 non-null float64
BVP_mean           785 non-null float64
BVP_std            785 non-null float64
BVP_min            785 non-null float64
BVP_max            785 non-null float64
ECG_mean           785 non-null float64
ECG_std          

In [32]:
pre.to_csv('fresh_start/60s_window_wrist_chest.csv',index=False)